# Final Capstone Report

## by Daniel da Cruz

### (Side note: Due to Foursquare API not functioning. The project has been adapted to compensate for the server-side error. The project below is focused on extracting data, preparing data, analyzing the data as well as providing a conclusion from the data.) :)

# The Business Problem 

Mr. Ronald is a 56 year old restaurenteer. He has been involved in the restaurant business since the age of 16. He has been quite succesful, and has built two businesses in the small town of Dundas, Ontario. 

His restaurants are part of a franchise called, "Ronald's Express." Mr. Ronald is the owner and founder of the franchise. Despite not having techincal background, he has identified a factors he believes has made his business a success. The menu is entirely designed by him. From all the years in the industry, his recipes have been created and refined to make his food the most delicious in his town. 

He sells burgers, tacos and pizzas using his very best recipes and cooking style. It is a town favourite. He know believes that Ronald's Express is ready to expand its business into Toronto, Ontario. The franchise is entirely family-owned, and he would like to pass it down to his sons when he retires. This will allow him to grow the business for many years to come. 

One of the key factors is opening a restaurant is the location. After a few failed busineses prior to Ronald's Expres, he realized that the location should have "plenty of feet." He is implying that the business need to be placed in a position to encounter a lot of people. He has also identified that his main customers are middle class people. He noticed that most of his customers are not exaclty "well off."

Ronald has been to Toronto a couple of times but its quite a big place. He has no idea where to begin looking for a place. He was told by his sons that he main need to find a data scientist to help with his problem. He has contacted me for assistance. 

# The Data and Methodology

Due to Toronto being a world-class city, it keeps all census data on its website www.toronto.ca. This website contains all the data required to understand the city, and how it functions. The most recent census data will be used as part of the analysis. 

The neighborhood data for Toronto is available as an Excel spreadsheet and all unnecessary data was removed. The data was then adjusted so that it could be used for analyzing. The three most important coloumns of data, with respect to the project, were the original population size of the Toronto neighborhoods, the area size of the neighborhood and the average family income. From the data, will create a new column that will give us the population density. 

We created the population density data by taking the population size and dividing it by the area size. This is justifiable deducation and will tell us how many people are present per square kilometre. This is what Mr. Ronald was talking about without being able to fully explain what he meant. We can report this figure to him, and provide a recommendation for where to start his search. 

We will then focus on the average family income. Mr. Ronald had explained that both the lower class and upper class economic brackets did not seem to make up his customer base. He beleives that the middle can both afford his products, and also enjoy it most. These are rough estimations on his parts and we will like to go by his word. 

After doing a statistical analysis, we will be able to report the mean and median incomes for the city of Toronto. This will give us a reference point for finding the middle class. It will also give us information on which towns to avoid. Financial metrics tend to have skewed data and we will likely have to use a median average to get a good reference for the middle class. 

After extracting and cleaning data, we will do a statistical analysis and report the findings using Plotly. We will plot the data on graphs depending on how will be best present the findings. 
